# Arcee AI Caller Model Demo with Together AI

This notebook demonstrates how to use the [Arcee AI Caller model](https://api.together.ai/models/arcee-ai/caller) through the Together AI API for function calling and tool use.


In [ ]:
## Install Required Packages

!pip install -U pip
!pip install -qU yfinance together

In [2]:
## Import Libraries

import json
import os
import pprint

import yfinance
from IPython.display import Markdown, clear_output, display
from together import Together

In [ ]:
## Initialize API Client

# Initialize the Together client
api_key = os.environ.get("TOGETHER_API_KEY")

# Verify API key is available
if api_key is None:
    print("⚠️ TOGETHER_API_KEY environment variable not found. Please set it before proceeding.")
    print("You can set it with: import os; os.environ['TOGETHER_API_KEY'] = 'your_api_key_here'")
else:
    # Create a client instance
    client = Together(api_key=api_key)
    print("✅ API key loaded successfully and client initialized")

In [4]:
## Define Tool Functions

def get_stock_price(company_name, stock_symbol):
    """Get the current stock price for a company."""
    try:
        stock = yfinance.Ticker(stock_symbol)
        price = stock.history(period="1d")["Close"].values[0]
        return f"The last closing price of {company_name} ({stock_symbol}) was ${price:.2f}."
    except Exception as e:
        return f"Error retrieving stock price for {company_name} ({stock_symbol}): {str(e)}"


def get_ceo_name(company_name, stock_symbol):
    """Get the CEO name and title for a company."""
    try:
        stock = yfinance.Ticker(stock_symbol)
        info = stock.info
        ceo = info["companyOfficers"][0]["name"]
        return f"The CEO of {company_name} is {ceo}. The full job title is {info['companyOfficers'][0]['title']}."
    except Exception as e:
        return f"Error retrieving CEO information for {company_name} ({stock_symbol}): {str(e)}"


def get_company_summary(company_name, stock_symbol):
    """Get a detailed business summary for a company."""
    try:
        stock = yfinance.Ticker(stock_symbol)
        summary = stock.info["longBusinessSummary"]
        return summary
    except Exception as e:
        return f"Error retrieving company summary for {company_name} ({stock_symbol}): {str(e)}"

In [ ]:
## Test Function: Stock Price

get_stock_price("Amazon", "AMZN")

In [ ]:
## Test Function: CEO Name

get_ceo_name("Ford", "F")

In [ ]:
## Test Function: Company Summary

get_company_summary("Verizon", "VZ")

In [8]:
## Define Tool Schemas for Function Calling

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": """Retrieves the most recent closing stock price that can answer questions about:
- Current stock price
- Recent stock performance
- Company market value
- Share price information

Use this function for questions like:
- "What's the stock price of [company]?"
- "How much does [company] stock cost?"
- "What's [company]'s current share price?"
- "How much are shares of [company] trading for?"
- "What's the latest price for [stock symbol]?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., 'Apple Inc.', 'Tesla, Inc.', 'McDonald's Corporation')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'AAPL', 'TSLA', 'MCD')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_ceo_name",
            "description": """Retrieves current leadership information that can answer questions about:
- CEO name and title
- Top executive information
- Company leadership
- Management details

Use this function for questions like:
- "Who is the CEO of [company]?"
- "Who runs [company]?"
- "What's the name of [company]'s chief executive?"
- "Who is in charge of [company]?"
- "Who is [company]'s leader?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., 'Microsoft Corporation', 'Meta Platforms, Inc.', 'Ford Motor Company')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'MSFT', 'META', 'F')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_company_summary",
            "description": """Retrieves a detailed business summary that can answer questions about:
- Products and services offered
- Target markets and industries served
- Manufacturing and production capabilities
- Business divisions and segments
- Geographic operations
- Company history and background
- Competitive position
- Core business activities

Use this function for questions like:
- "Does [company] make [product]?"
- "What industries does [company] serve?"
- "What are [company]'s main products?"
- "Does [company] operate in [industry/market]?"
- "What does [company] do?"
- "Tell me about [company]'s business"
- "What kind of company is [company]?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., '3M Company', 'Johnson & Johnson', 'The Coca-Cola Company')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'MMM', 'JNJ', 'KO')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
]

In [14]:
## Function to Call Tools with Caller Model

def call_tools(user_prompt, max_tokens=128):
    """
    Call the Arcee AI Caller model to determine which function to call based on the user prompt.
    
    Args:
        user_prompt: The user's question or request
        max_tokens: Maximum number of tokens to generate
        
    Returns:
        List of results from the called functions or None if no function was called
    """
    try:
        response = client.chat.completions.create(
            model="arcee-ai/caller",
            messages=[{"role": "user", "content": user_prompt}],
            tools=tools,
            tool_choice="auto",
            max_tokens=max_tokens,
        )

        tool_calls = response.choices[0].message.tool_calls
        results = []

        # Check if there are any tool calls
        if tool_calls:
            for tool_call in tool_calls:
                # Extract function name and arguments
                function_name = tool_call.function.name
                arguments_json = tool_call.function.arguments
                arguments_dict = json.loads(arguments_json)

                if function_name in globals():
                    # Get the function object based on its name
                    function_to_call = globals()[function_name]

                    print(f"Calling {function_name} with arguments: {arguments_dict}")

                    # Call the function with unpacked keyword arguments
                    result = function_to_call(**arguments_dict)
                    results.append(result)
                else:
                    print(f"Function {function_name} not found in the global namespace.")
                    results.append(None)

            return results
        else:
            # No tool call: print the generated response
            print("No tool call")
            return None
            
    except Exception as e:
        print(f"Error in call_tools: {str(e)}")
        return [f"Error: {str(e)}"]

In [20]:
## Function to Call Tools and Generate Response with LLM

def call_tools_and_invoke_model(model, user_prompt, max_tokens=1024):
    """
    Two-step process:
    1. Call the Caller model to determine which function to use
    2. Pass the function results to a larger model to generate a comprehensive response
    
    Args:
        model: The model to use for generating the final response
        user_prompt: The user's question or request
        max_tokens: Maximum number of tokens to generate
        
    Returns:
        Streaming response from the model
    """
    try:    
        # Step 1: Call the tools
        tool_results = call_tools(user_prompt)

        # Convert list of results to string if we got multiple results
        tool_results_str = (
            "\n".join(tool_results) if isinstance(tool_results, list) else tool_results
        )

        # Step 2: Generate comprehensive response with the model
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": """You are an expert financial and business analyst providing precise, well-researched answers.

Your capabilities:
- Analyze company information, financials, and market data
- Compare companies and their products/services
- Provide insights about business strategies and market positions
- Explain industry trends and competitive dynamics

Guidelines:
1. Always start with analyzing the tool result if available
2. Structure your response clearly using markdown
3. Be detailed and thorough
4. Include relevant context from your knowledge
5. Support claims with data and sources when possible
6. Stay neutral and objective

Required sections:
* Tool information - Did you use the tool or not?
* Direct Answer - Address the main question first.
* Analysis - Provide context and deeper insights
* Resources - if applicable, include for all companies:
   - Top-level company website
   - Yahoo Finance page (if the company is listed)
   - URL for company SEC filings (if the company is publicly traded)
   - URL for Google search based on the user question

If the tool result is unavailable or insufficient, acknowledge this first and proceed with your best analysis based on available knowledge.""",
                },
                {
                    "role": "user",
                    "content": f"""
                        question: {user_prompt}
                        tool results: {tool_results_str}
                    """,
                },
            ],
            max_tokens=max_tokens,
            stream=True,
        )
        return response
        
    except Exception as e:
        print(f"Error in call_tools_and_invoke_model: {str(e)}")
        # Return a dummy response object with an error message
        class DummyResponse:
            def __iter__(self):
                yield self
                
            @property
            def choices(self):
                class Choice:
                    @property
                    def delta(self):
                        class Delta:
                            @property
                            def content(self):
                                return f"Error: {str(e)}"
                        return Delta()
                return [Choice()]
                
        return DummyResponse()

In [21]:
## Helper Function to Display Streaming Response

def print_streaming_response(response):
    """
    Display a streaming response from the model with Markdown formatting.
    
    Args:
        response: Streaming response from the model
    """
    try:
        num_tokens = 0
        content = ""
        for message in response:
            if hasattr(message, 'choices') and len(message.choices) > 0:
                num_tokens += 1
                chunk = message.choices[0].delta.content
                if chunk:
                    content += chunk
                    clear_output(wait=True)
                    display(Markdown(content))

        print(f"\n\nNumber of tokens: {num_tokens}")
    except Exception as e:
        print(f"Error displaying streaming response: {str(e)}")
        if content:
            display(Markdown(content))

In [ ]:
## Set User Query

# Choose one of the example queries by uncommenting it
user_prompt = "Who's the CEO of General Motors?"
#user_prompt = "What's the last price of McDonald's?"
#user_prompt = "Does 3M make filtration products for the automotive industry?"
#user_prompt = "On which products do Procter and Gamble and Johnson & Johnson compete the most?"
#user_prompt = "What's the population of the capital of New Zealand?"

print(f"Selected query: '{user_prompt}'")
print("You can modify this cell to try different queries")

In [ ]:
## Step 1: Call the Tool Function

print("Calling the Arcee AI Caller model to determine which function to use...")

# Call the Caller model to determine which function to use
response = call_tools(user_prompt)

# Display the function result
print("\nFunction result:")
if response:
    pprint.pprint(response)
else:
    print("No function was called or no result was returned.")

In [ ]:
## Step 2: Generate Comprehensive Response with Virtuoso Large

print("Calling the Arcee AI Virtuoso Large model to generate a comprehensive response...")
print("This will stream the response as it's being generated...\n")

# Call the Virtuoso Large model with the function result
response = call_tools_and_invoke_model(model="arcee-ai/virtuoso-large", user_prompt=user_prompt)

# Display the streaming response
print_streaming_response(response)

## Conclusion

This notebook demonstrated how to use the Arcee AI models on Together AI to:
1. Call the specialized Caller model to determine which function to use based on a user query
2. Execute the appropriate function to retrieve information
3. Pass the function results to the Virtuoso Large model to generate a comprehensive response

You can modify the user prompt and try different questions to see how the models handle various types of queries.
